<a href="https://colab.research.google.com/github/MizanMustakim/Big_Data_project_summer_semester/blob/main/Big_data_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Collection work

**Import the libraries**

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Collecting DATA_1**

In [3]:
url = "https://www.worldometers.info/coronavirus/?utm_campaign=homeAdvegas1?%22%20%5Cl%22countries"
data = requests.get(url)
soup = BeautifulSoup(data.text,'html5lib')

In [4]:
covid_dict = {}
div = soup.find_all("div", {"id": "ExplorerContainer"})
for i in div:
  content_div = i.find("div",{"class":"tableTab"})
  covid_dict[i.find("h1").text.replace(":","").strip()] = content_div.find("span").text.strip()
print(covid_dict)

{}


In [5]:
tables = soup.find_all('table')
table_header = tables[0].find_all('th')
table_head = []
for i in range(15):
  if i != 0:
    table_head.append(table_header[i].text.replace("\n","").replace("\xa0",""))
print(table_head)

['Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths', 'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical', 'TotCases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop', 'Population']


In [6]:
Covid_data = pd.DataFrame(columns=table_head)
for row in tables[0].tbody.find_all('tr'):
  col = row.find_all('td')
  if (col != []):
    country = col[1].text.strip()
    totalCases = col[2].text.strip()
    newCases = col[3].text.strip()
    totalDeaths = col[4].text.strip()
    newDeaths = col[5].text.strip()
    totalRecovered = col[6].text.strip()
    newRecovered = col[7].text.strip()
    activeCases = col[8].text.strip()
    serious = col[9].text.strip()
    totalCases_per_m = col[10].text.strip()
    deaths = col[11].text.strip()
    totalTests = col[12].text.strip()
    tests_per_m = col[13].text.strip()
    population = col[14].text.strip()
    Covid_data = Covid_data.append({"Country,Other":country,"TotalCases":totalCases,"NewCases":newCases,
                                    "TotalDeaths":totalDeaths,"NewDeaths":newDeaths,"TotalRecovered":totalRecovered,
                                    "NewRecovered":newRecovered,"ActiveCases":activeCases,"Serious,Critical":serious,
                                    "TotCases/1M pop":totalCases_per_m,"Deaths/1M pop":deaths,"TotalTests":totalTests,
                                    "Tests/1M pop":tests_per_m,"Population":population},ignore_index=True)

In [7]:
Covid_data.drop(Covid_data.index[:7],inplace=True)
Covid_data.to_csv("Covid_data.csv",index=False)

**Collecting DATA_2**

In [8]:
!wget --no-check-certificate \
  https://covid.ourworldindata.org/data/owid-covid-data.csv \
  -O /content/owid-covid-data.csv

--2021-09-09 10:51:48--  https://covid.ourworldindata.org/data/owid-covid-data.csv
Resolving covid.ourworldindata.org (covid.ourworldindata.org)... 104.21.233.137, 104.21.233.138, 2606:4700:3038::6815:e98a, ...
Connecting to covid.ourworldindata.org (covid.ourworldindata.org)|104.21.233.137|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘/content/owid-covid-data.csv’

/content/owid-covid     [  <=>               ]  29.52M   105MB/s    in 0.3s    

2021-09-09 10:51:48 (105 MB/s) - ‘/content/owid-covid-data.csv’ saved [30958093]



# Data Processing Work

**Install PySpark**

In [9]:
!pip install pyspark

**Importing Libraries**

In [10]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains

**Loading and Checking data by pandas**

In [11]:
df_vaccine = pd.read_csv("/content/owid-covid-data.csv")
df_vaccine.head(4)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,0.025,0.025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,0.025,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,0.025,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,0.025,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN


In [12]:
df_vaccine.tail(4)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
115441,ZWE,Africa,Zimbabwe,2021-09-05,125550.0,125.0,159.000,4482.0,16.0,11.571,8318.883,8.282,10.535,296.975,1.060,0.767,0.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4431599.0,2713248.0,1718351.0,NaN,31353.0,41964.0,29.36,17.98,11.39,NaN,2781.0,78.7,15092171.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN
115442,ZWE,Africa,Zimbabwe,2021-09-06,125671.0,121.0,155.714,4493.0,11.0,11.000,8326.900,8.017,10.318,297.704,0.729,0.729,0.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41445.0,NaN,NaN,NaN,NaN,2746.0,78.7,15092171.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN
115443,ZWE,Africa,Zimbabwe,2021-09-07,125796.0,125.0,146.143,4503.0,10.0,12.000,8335.183,8.282,9.683,298.367,0.663,0.795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4493943.0,2750381.0,1743562.0,NaN,NaN,39160.0,29.78,18.22,11.55,NaN,2595.0,NaN,15092171.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN
115444,ZWE,Africa,Zimbabwe,2021-09-08,125931.0,135.0,138.714,4517.0,14.0,11.286,8344.128,8.945,9.191,299.294,0.928,0.748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15092171.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN


**Get the coloumn list of DATA_2**

In [13]:
df_vaccine.columns

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'new_tests', 'total_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed',
       't

**Processing DATA with PySpark**

In [14]:
spark = SparkSession.builder.appName('CSV reader').getOrCreate()

#extract
df = '/content/owid-covid-data.csv'
sdfData = spark.read.csv(df, header=True, sep=",").cache()
Name = sdfData.groupBy('Continent').count()
print(Name.show())

+-------------+-----+
|    Continent|count|
+-------------+-----+
|       Europe|27112|
|       Africa|29353|
|         null| 5290|
|North America|14759|
|South America| 6812|
|      Oceania| 5159|
|         Asia|26960|
+-------------+-----+

None


In [15]:
sdfData.printSchema()

root
 |-- iso_code: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- location: string (nullable = true)
 |-- date: string (nullable = true)
 |-- total_cases: string (nullable = true)
 |-- new_cases: string (nullable = true)
 |-- new_cases_smoothed: string (nullable = true)
 |-- total_deaths: string (nullable = true)
 |-- new_deaths: string (nullable = true)
 |-- new_deaths_smoothed: string (nullable = true)
 |-- total_cases_per_million: string (nullable = true)
 |-- new_cases_per_million: string (nullable = true)
 |-- new_cases_smoothed_per_million: string (nullable = true)
 |-- total_deaths_per_million: string (nullable = true)
 |-- new_deaths_per_million: string (nullable = true)
 |-- new_deaths_smoothed_per_million: string (nullable = true)
 |-- reproduction_rate: string (nullable = true)
 |-- icu_patients: string (nullable = true)
 |-- icu_patients_per_million: string (nullable = true)
 |-- hosp_patients: string (nullable = true)
 |-- hosp_patients_per_million

In [16]:
#transform
sdfData.registerTempTable("cases")
output =  spark.sql('SELECT * from cases')



schema = StructType() \
      .add("Name",StringType(),True) \
      .add("continent",StringType(),True) \

df_with_schema = spark.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load("/content/owid-covid-data.csv")
df_with_schema.printSchema()

root
 |-- Name: string (nullable = true)
 |-- continent: string (nullable = true)



In [17]:
output = spark.sql('SELECT * from cases WHERE new_cases > 150000' )
output.show(10)

+--------+---------+--------+----------+-----------+---------+------------------+------------+----------+-------------------+-----------------------+---------------------+------------------------------+------------------------+----------------------+-------------------------------+-----------------+------------+------------------------+-------------+-------------------------+---------------------+---------------------------------+----------------------+----------------------------------+---------+-----------+------------------------+----------------------+------------------+-------------------------------+-------------+--------------+-----------+------------------+-----------------+-----------------------+--------------+----------------+-------------------------+------------------------------+-----------------------------+-----------------------------------+--------------------------+-------------------------------------+----------------+------------+------------------+----------+-----

In [18]:
output = spark.sql('SELECT COUNT(*) as total, continent from cases GROUP BY continent')
output.show()

+-----+-------------+
|total|    continent|
+-----+-------------+
|27112|       Europe|
|29353|       Africa|
| 5290|         null|
|14759|North America|
| 6812|South America|
| 5159|      Oceania|
|26960|         Asia|
+-----+-------------+



In [19]:
sdfData.registerTempTable("total_vaccinations")
sdfData.where(sdfData.total_vaccinations.isNull())
output =  spark.sql('SELECT location, total_vaccinations from total_vaccinations WHERE date = "2021-03-25"')
output.show(45)

+--------------------+------------------+
|            location|total_vaccinations|
+--------------------+------------------+
|         Afghanistan|              null|
|              Africa|         9709302.0|
|             Albania|           59348.0|
|             Algeria|              null|
|             Andorra|              null|
|              Angola|              null|
|            Anguilla|              null|
| Antigua and Barbuda|           26424.0|
|           Argentina|         3365839.0|
|             Armenia|              null|
|                Asia|       209396306.0|
|           Australia|          408410.0|
|             Austria|         1509953.0|
|          Azerbaijan|              null|
|             Bahamas|              null|
|             Bahrain|          707980.0|
|          Bangladesh|         5139456.0|
|            Barbados|           62802.0|
|             Belarus|              null|
|             Belgium|         1752182.0|
|              Belize|            

In [20]:
output = spark.sql('SELECT * from cases WHERE new_cases > 150000' )
output.show()

+--------+---------+--------+----------+-----------+---------+------------------+------------+----------+-------------------+-----------------------+---------------------+------------------------------+------------------------+----------------------+-------------------------------+-----------------+------------+------------------------+-------------+-------------------------+---------------------+---------------------------------+----------------------+----------------------------------+---------+-----------+------------------------+----------------------+------------------+-------------------------------+-------------+--------------+-----------+------------------+-----------------+-----------------------+--------------+----------------+-------------------------+------------------------------+-----------------------------+-----------------------------------+--------------------------+-------------------------------------+----------------+------------+------------------+----------+-----

In [21]:
output = spark.sql('SELECT continent, COUNT(*) as total from cases GROUP BY continent')
output.show()

+-------------+-----+
|    continent|total|
+-------------+-----+
|       Europe|27112|
|       Africa|29353|
|         null| 5290|
|North America|14759|
|South America| 6812|
|      Oceania| 5159|
|         Asia|26960|
+-------------+-----+



In [22]:
output.write.format('json').save('filtered.json')